In [1]:
import pickle
import deepchem as dc
import sys
sys.path.insert(0,'../')
import src
import requests
import json
import pandas as pd
import numpy as np

from src.dc_utils import data_to_json, format_request, response_to_csv

# Demo 1: get metadata

In [2]:
host1 = 'http://localhost:5000/'

response = requests.get(host1)
print('metadata:\n{}'.format(response.text))


metadata:
{"Release":"beta"}



# Demo 2: Predict on data from DeepChem

In [3]:
tox21_tasks_2, tox21_datasets_2, transformers_2 = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset_2, valid_dataset_2, test_dataset_2 = tox21_datasets_2

In [4]:
test_data = data_to_json(test_dataset_2)
data_json = format_request(test_data)

host2 = f'{host1}/predict'
response = requests.post(host2, data=data_json)
print(response.text[:1000])

[["estrogen receptor alpha, LBD (ER, LBD): inactive","estrogen receptor alpha, full (ER, full): inactive","aromatase: inactive","aryl hydrocarbon receptor (AhR): inactive","androgen receptor, full (AR, full): inactive","androgen receptor, LBD (AR, LBD): inactive","peroxisome proliferator-activated receptor gamma (PPAR-gamma): inactive","nuclear factor (erythroid-derived 2)-like 2/antioxidant responsive element (Nrf2/ARE): inactive","heat shock factor response element (HSE): inactive","ATAD5: inactive","mitochondrial membrane potential (MMP): inactive","p53: inactive"],["estrogen receptor alpha, LBD (ER, LBD): active","estrogen receptor alpha, full (ER, full): inactive","aromatase: active","aryl hydrocarbon receptor (AhR): active","androgen receptor, full (AR, full): active","androgen receptor, LBD (AR, LBD): active","peroxisome proliferator-activated receptor gamma (PPAR-gamma): active","nuclear factor (erythroid-derived 2)-like 2/antioxidant responsive element (Nrf2/ARE): active","hea

# Demo 3: Predict on molecule from SMILES representation

In [5]:
molecules = ['CC(C)(c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1)c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1',
       'Cc1cc(C(C)(C)C)c(O)c(C)c1Cn1c(=O)n(Cc2c(C)cc(C(C)(C)C)c(O)c2C)c(=O)n(Cc2c(C)cc(C(C)(C)C)c(O)c2C)c1=O',
       'Cc1nnc(-c2ccccc2)c(=O)n1N', 'N=C(N)NCC1COc2ccccc2O1',
       'Cc1cccc(C)c1NC(=O)NC1=CCCN1C', 'c1csc(C2(N3CCCCC3)CCCCC2)c1',]

featurizer = dc.feat.ConvMolFeaturizer()
conv_mols = featurizer(molecules)
dataset = dc.data.NumpyDataset(conv_mols, ids=molecules, n_tasks=12)
dataset.tasks = tox21_tasks_2

dataset_json = format_request(data_to_json(dataset))
response_2 = requests.post(host2, data=dataset_json)

In [6]:
response_to_csv(response_2, molecules, 'results.csv')